In [ ]:
# All necessary imports
from __future__ import print_function, division

import os
import time
import copy
import pandas as pd
from skimage import io, transform
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import datasets, models, transforms

from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt
%matplotlib inline


In [ ]:
!unzip '/content/train_X.zip'

Archive:  /content/train_X.zip
replace train_X.npy? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: train_X.npy             



In [ ]:
!unzip '/content/train_X.zip'

Archive:  /content/train_X.zip
replace train_X.npy? [y]es, [n]o, [A]ll, [N]one, [r]ename: r
new name: training_X
  inflating: training_X              


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/', force_remount=True)

Mounted at /content/gdrive/


In [ ]:
#LOAD DATA
# Lists of hyperparameters here
# Feel free to add more if needed
batch_size = 128
validate_every = 1
lr = 1e-3
weight_decay = 5e-3
epochs = 20
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

file_path = "/content"
x_train = torch.tensor(np.load(file_path + "/train_X.npy"))

x_np = np.load(file_path + "/train_X.npy")
x_train = x_train.view(55000, 3, 64, 64)
x_validation = torch.tensor(np.load(file_path + "/valid_X.npy"))
validation_np = np.load(file_path + "/valid_X.npy")

x_validation = x_validation.view(5000, 3, 64, 64)
print("Images:", x_train.shape, x_validation.shape)

y_train = torch.tensor(np.load(file_path + "/train_Y.npy")).type(torch.LongTensor)
y_validation = torch.tensor(np.load(file_path + "/valid_Y.npy")).type(torch.LongTensor)
print("Y train and validation", y_train.shape, y_validation.shape)
      
train_bboxes = torch.Tensor(np.load(file_path + "/train_bboxes.npy")).type(torch.LongTensor)
valid_bboxes = torch.Tensor(np.load(file_path + "/valid_bboxes.npy")).type(torch.LongTensor)


print("Bounding boxes: ",train_bboxes.shape, valid_bboxes.shape)

Images: torch.Size([55000, 3, 64, 64]) torch.Size([5000, 3, 64, 64])
Y train and validation torch.Size([55000, 2]) torch.Size([5000, 2])
Bounding boxes:  torch.Size([55000, 2, 4]) torch.Size([5000, 2, 4])


In [ ]:
!git clone https://github.com/ultralytics/yolov5  # clone
%cd yolov5
%pip install -qr requirements.txt  # install

import torch
import utils
display = utils.notebook_init()  # checks

YOLOv5 🚀 v6.2-228-g6ae3dff Python-3.7.15 torch-1.12.1+cu113 CUDA:0 (Tesla T4, 15110MiB)


Setup complete ✅ (2 CPUs, 12.7 GB RAM, 22.6/78.2 GB disk)


In [ ]:
import PIL
from PIL import Image

for i in range(55000):
  src_img = x_np[i].squeeze().reshape((64, 64, 3)).astype(np.uint8)
  im = Image.fromarray(src_img)
  im.save("/content/train_data/images/train/" + str(i) + ".png")

for i in range(5000):
  src_img = validation_np[i].squeeze().reshape((64, 64, 3)).astype(np.uint8)
  im = Image.fromarray(src_img)
  im.save("/content/train_data/images/val/" + str(i) + ".png")

In [ ]:
def convert(y1, x1, y2, x2, image_w, image_h):
    return [((x2 + x1)/(2*image_w)) * 0.4375, ((y2 + y1)/(2*image_h))* 0.4375, (x2 - x1)/image_w* 0.4375, (y2 - y1)/image_h* 0.4375]

##LABELS

In [ ]:
x_bboxes = []
import tensorflow as tf

for i in range(55000):
  w= 28
  h= 28
  d1 = train_bboxes[i][0].numpy()
  d2 = train_bboxes[i][1].numpy()
  _d1 =list(convert(d1[0], d1[1], d1[2], d1[3], w, h))
  _d2 = list(convert(d2[0], d2[1], d2[2], d2[3], w, h))
  a, b = y_train[i][0].view(1).numpy(), y_train[i][1].view(1).numpy()
  _d1.insert(0, a[0])
  _d2.insert(0, b[0])

  str1 = " ".join(map(str,_d1))
  fin = str1 + "\n" + " ".join(map(str, _d2))
  x_bboxes.append(fin)
  
  # just to test the 52000th img
  if i == 54999:
    print(x_bboxes[i])

len(x_bboxes)

2 0.34375 0.640625 0.4375 0.4375
7 0.671875 0.734375 0.4375 0.4375


55000

In [ ]:
val_bboxes = []
import tensorflow as tf

for i in range(5000):
  w= 28
  h= 28
  d1 = valid_bboxes[i][0].numpy()
  d2 = valid_bboxes[i][1].numpy()
  _d1 =list(convert(d1[0], d1[1], d1[2], d1[3], w, h))
  _d2 = list(convert(d2[0], d2[1], d2[2], d2[3], w, h))

  a, b = y_validation[i][0].view(1).numpy(), y_validation[i][1].view(1).numpy()

  _d1.insert(0, a[0])
  _d2.insert(0, b[0])

  str1 = " ".join(map(str,_d1))
  fin = str1 + "\n" + " ".join(map(str, _d2))
  val_bboxes.append(fin)

  # just to test the 4900th img
  if i == 0:
    print(val_bboxes[i])
    # plt.imshow(validation_images[i])

len(val_bboxes)

3 0.578125 0.703125 0.4375 0.4375
5 0.78125 0.734375 0.4375 0.4375


5000

In [ ]:
for i in range(5000):
  with open(r'/content/train_data/labels/val/' + str(i) + '.txt', 'w') as fp:

    # write each item on a new line
      fp.write("%s\n" % val_bboxes[i])
print('Done')

Done


In [ ]:
for i in range(55000):
  with open(r'/content/train_data/labels/train/' + str(i) + '.txt', 'w') as fp:

    # write each item on a new line
      fp.write("%s\n" % x_bboxes[i])
print('Done')

Done


In [ ]:
!python train.py --img 640 --batch 40 --epochs 30 --data custom_data.yaml --weights yolov5s.pt 

train: weights=yolov5s.pt, cfg=, data=custom_data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=30, batch_size=40, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
Command 'git fetch origin' timed out after 5 seconds
YOLOv5 🚀 v6.2-228-g6ae3dff Python-3.7.15 torch-1.12.1+cu113 CUDA:0 (Tesla T4, 15110MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv_h=0.0, hsv_s=0.0, hsv_v=0.0, degrees=